### Intersecting AIS Features

#### Date created: 11/21/2024
#### Clip all AIS layers for 2023 to the SoCal BOEM aliquot grid and export
#### AIS lines were created by Brian Free using points from Marine Cadastre

In [5]:
#import system modules and set environment
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.sa import *
arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'C:/Users/Eliza.Carter/Documents/Projects/California/socal_ais'

In [6]:
#define spatial reference (EPSG for NAD 83 UTM Zone 11N)
spatial_ref = arcpy.SpatialReference(26911)

boem_grid = 'data/b_intermediate_data/socal_boem_grid.shp'

In [7]:
# Paths
gpkg_path = 'data/a_raw_data/wc_ais_transect.gpkg'
output_folder = 'data/b_intermediate_data/'

# List GPKG layers
gpkg_description = arcpy.Describe(gpkg_path)
layer_names = [layer.name for layer in gpkg_description.children]

# Loop through each layer
for layer_name in layer_names:
    layer_path = f"{gpkg_path}/{layer_name}"
    
    # Extract the last part of the layer name for output file naming
    output_filename = layer_name.split('.')[-1] + '_intersect.shp'
    output_path = f"{output_folder}{output_filename}"

    # Create a feature layer
    arcpy.management.MakeFeatureLayer(layer_path, "ais_transects")

    # Select features that intersect the grid
    arcpy.management.SelectLayerByLocation(
        in_layer="ais_transects",
        overlap_type="INTERSECT",
        select_features=boem_grid,
        search_distance=None,
        selection_type="NEW_SELECTION"
    )

    # Copy selected features to a new shapefile
    arcpy.management.CopyFeatures("ais_transects", output_path)

    # Clean up
    arcpy.management.Delete("ais_transects")

    # Project the intersecting AIS transects
    projected_path = output_path.replace("_intersect.shp", "_socal.shp")
    arcpy.management.Project(output_path, projected_path, spatial_ref)

    # Clean up intermediate files
    arcpy.management.Delete(output_path)
